# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phạm Nguyễn Quốc Thanh
- MSSV: 21127428
- Lớp: 21CLC08

# Import

In [2205]:
import pandas as pd
import numpy as np
from IPython.display import Markdown as md
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
# Import thêm dữ thư viện nếu cần
#1: Male
#2: Female

# Đọc dữ liệu

In [2206]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (X_train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)
X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)
first_11_data_train = train.iloc[:, :11]
first_11_data_test = test.iloc[:, :11]
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [2243]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel())) 

def format_float(x):
    return '{:.3f}'.format(x)

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [2244]:
# Phần code cho yêu cầu 1a
#Change Gender colum values from 1 to 0 and 2 to 1
lr_1a = OLSLinearRegression().fit(first_11_data_train, y_train)

In [2245]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr_1a.predict(first_11_data_test)

#Library
# print("MAE: ", mean_absolute_error(y_test.to_numpy().reshape(-1, 1), y_hat.reshape(-1, 1)))
# print(lr_1a.coef_)
# print(lr_1a.intercept_)

#Self
print("MAE trên tập kiểm tra: ", mae(y_test.to_numpy().reshape(-1, 1), y_hat))
weights = lr_1a.get_params()
print("Weights:\n", weights)

MAE trên tập kiểm tra:  104863.77754033124
Weights:
 0    -22756.512821
1       804.503156
2      1294.654565
3    -91781.897530
4     23182.388679
5      1437.548672
6     -8570.661985
7       147.858299
8       152.888476
9       117.221846
10    34552.286221
dtype: float64


In [2246]:
first_11_columns = X_train.columns[:-1].to_numpy()
results = zip(first_11_columns, lr_1a.get_params())

display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = " + " + ".join(f"{w:.3f} x {f}" for f, w in results)))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = -22756.513 x Gender + 804.503 x 10percentage + 1294.655 x 12percentage + -91781.898 x CollegeTier + 23182.389 x Degree + 1437.549 x collegeGPA + -8570.662 x CollegeCityTier + 147.858 x English + 152.888 x Logical + 117.222 x Quant + 34552.286 x Domain

# Xáo trộn dữ liệu 1 lần cho toàn bộ các câu còn lại

In [2247]:
train = train.sample(frac=1)
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [2260]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
kfold = KFold(n_splits=10, shuffle=False)
models_train = [X_train.loc[:, 'conscientiousness'].to_numpy().reshape(-1, 1), X_train.loc[:, 'agreeableness'].to_numpy().reshape(-1, 1), X_train.loc[:, 'extraversion'].to_numpy().reshape(-1, 1), X_train.loc[:, 'openess_to_experience'].to_numpy().reshape(-1, 1), X_train.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)]
models_test = [X_test.loc[:, 'conscientiousness'].to_numpy().reshape(-1, 1), X_test.loc[:, 'agreeableness'].to_numpy().reshape(-1, 1), X_test.loc[:, 'extraversion'].to_numpy().reshape(-1, 1), X_test.loc[:, 'openess_to_experience'].to_numpy().reshape(-1, 1), X_test.loc[:, 'nueroticism'].to_numpy().reshape(-1, 1)]
models_name = ['conscientiousness', 'agreeableness', 'extraversion', 'openess_to_experience', 'neuroticism']

average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train, cv=kfold, scoring='neg_mean_absolute_error')).mean() for model in models_train]
best_model_index = average_maes.index(min(average_maes))

df = pd.DataFrame(list(zip(models_name, average_maes)), columns =['Mô hình với 1 đặc trưng', 'MAE'])
df.sort_values(by=['MAE'], inplace=True)
df.to_clipboard()
print(df)
#Nueroticism là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng            MAE
4             neuroticism  299248.592105
1           agreeableness  300918.330485
3   openess_to_experience  303138.560006
0       conscientiousness  306115.229468
2            extraversion  306960.136728


In [2258]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
my_best_personality_feature_model = OLSLinearRegression().fit(models_train[best_model_index], y_train)

In [2259]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
X_Para = models_test[best_model_index]
y_hat = my_best_personality_feature_model.predict(X_Para)
print("MAE trên tập kiểm tra: ", mae(y_test, y_hat))
print(my_best_personality_feature_model.w)

MAE trên tập kiểm tra:  291019.693226953
[-56546.30375318]


In [2215]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = Neuroticism x -56546.304"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = Neuroticism x -56546.304

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [2261]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
kfold = KFold(n_splits=10, shuffle=False)
models_train = [X_train.loc[:, 'Quant'].to_numpy().reshape(-1, 1), X_train.loc[:, 'Logical'].to_numpy().reshape(-1, 1), X_train.loc[:, 'English'].to_numpy().reshape(-1, 1)]
models_test = [X_test.loc[:, 'Quant'].to_numpy().reshape(-1, 1), X_test.loc[:, 'Logical'].to_numpy().reshape(-1, 1), X_test.loc[:, 'English'].to_numpy().reshape(-1, 1)]
models_name = ['Quant', 'Logical', 'English']

average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train, cv=kfold, scoring='neg_mean_absolute_error')).mean() for model in models_train]
best_model_index = average_maes.index(min(average_maes))

df = pd.DataFrame(list(zip(models_name, average_maes)), columns =['Mô hình với 1 đặc trưng', 'MAE'])
df.sort_values(by=['MAE'], inplace=True)
df.to_clipboard()
print(df)
#Quant là đặc trưng tốt nhất

  Mô hình với 1 đặc trưng            MAE
0                   Quant  118144.208902
1                 Logical  120332.924373
2                 English  121948.949445


In [2262]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
my_best_skill_feature_model = OLSLinearRegression().fit(models_train[best_model_index], y_train)

In [2263]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
X_Para = models_test[best_model_index]
y_hat = my_best_skill_feature_model.predict(X_Para)
print("MAE trên tập kiểm tra: ", mae(y_test, y_hat))
print(my_best_skill_feature_model.w)

MAE trên tập kiểm tra:  106819.57761989674
[585.89538084]


In [2264]:
display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = Quant x 585.895"))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = Quant x 585.895

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [2265]:
# Trình bày các phần tìm ra mô hình
kfold = KFold(n_splits=10, shuffle=False)
models_train = []
models_test = []
for i in range(23):
    single_feature_model = X_train.iloc[:, i].to_numpy().reshape(-1, 1)
    models_train.append(single_feature_model)
    models_test.append(X_test.iloc[:, i])
models_train.append(X_train)
models_test.append(X_test)

average_maes = np.zeros(len(models_train))
average_maes = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model , y_train, scoring='neg_mean_absolute_error', cv=kfold).mean()) for model in models_train]
#Average weights for all models
average_weights = np.zeros(len(models_train))
average_weights = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model , y_train, scoring='neg_mean_absolute_error', cv=kfold).mean()) for model in models_train]
df = []
for i in range(23):
    row = []
    row.append(X_train.columns[i])
    row.append(average_maes[i])
    df.append(row)
df.append(['All', average_maes[23]])
df = pd.DataFrame(df, columns=['Feature', 'MAE'])
df = df.sort_values(by=['MAE'])
df.to_clipboard()
print(df) 

                  Feature            MAE
23                    All  110419.182513
9                   Quant  118144.208902
1            10percentage  118852.566509
2            12percentage  120033.097356
8                 Logical  120332.924373
5              collegeGPA  121531.961882
7                 English  121948.949445
3             CollegeTier  133475.597572
4                  Degree  137527.571455
0                  Gender  150416.337377
11    ComputerProgramming  156016.376342
10                 Domain  175539.142292
6         CollegeCityTier  251450.680674
12  ElectronicsAndSemicon  257427.229807
13        ComputerScience  268409.984478
16            TelecomEngg  290913.932167
14         MechanicalEngg  297415.809472
21            nueroticism  299248.592105
19          agreeableness  300918.330485
15         ElectricalEngg  301204.472375
22  openess_to_experience  303138.560006
18      conscientiousness  306115.229468
17              CivilEngg  306116.156725
20           ext

## Thử nghiệm, so sánh các mô hình

In [2224]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
kfold = KFold(n_splits=10, shuffle=False)

#Mô hình thứ nhất: Sử dụng 23 đặc trưng
first_model = X_train
first_model_test = X_test
first_model_name = 'Mô hình 23 đặc trưng'
#Mô hình thứ hai: Sử dụng các đặc trưng Logical, Quant, Domain và English
dropped_features = ['Degree']
second_model = X_train.drop(columns=dropped_features)
second_model_test = X_test.drop(columns=dropped_features)
second_model_name = 'Mô hình 22 đặc trưng (Degree))'

#Mô hình thứ ba:
dropped_features = ['MechanicalEngg', 'CivilEngg']
third_model = X_train.drop(columns=dropped_features)
third_model_test = X_test.drop(columns=dropped_features)
third_model_name = 'Mô hình 21 đặc trưng (Mech, Civil)'

models_train = [first_model, second_model, third_model]
models_test = [first_model_test, second_model_test, third_model_test]
models_name = [first_model_name, second_model_name, third_model_name]

average_mae = np.zeros(len(models_train))
average_mae = [np.abs(cross_val_score(LinearRegression(fit_intercept=False), model, y_train, cv=kfold, scoring='neg_mean_absolute_error').mean()) for model in models_train]
data = []
for i in range(len(models_train)):
    data.append([models_name[i], average_mae[i]])
df = pd.DataFrame(data, columns=['Mô hình', 'MAE'])
df.sort_values(['MAE'], inplace=True)
df.to_clipboard()
print(df)

                              Mô hình            MAE
1      Mô hình 22 đặc trưng (Degree))  110575.536943
2  Mô hình 21 đặc trưng (Mech, Civil)  110613.592179
0                Mô hình 23 đặc trưng  110761.273488


In [2222]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
best_model_index = np.argmin(average_mae)
my_best_model = LinearRegression(fit_intercept=False).fit(models_train[best_model_index], y_train)

In [2223]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
y_hat_best = my_best_model.predict(models_test[best_model_index])
print("MAE trên tập kiểm tra: ", mae(y_test, y_hat_best))
results = pd.DataFrame(list(zip(models_test[best_model_index].columns, np.round(my_best_model.coef_, 3))) )
print("Các hệ số tương ứng với các đặc trưng:\n",np.array(results).reshape(-1, 2))
results.to_clipboard()

MAE trên tập kiểm tra:  101582.13373031771
Các hệ số tương ứng với các features:
 [['Gender' -23699.803]
 ['10percentage' 875.683]
 ['12percentage' 1171.61]
 ['CollegeTier' -81235.349]
 ['collegeGPA' 1772.254]
 ['CollegeCityTier' -5421.824]
 ['English' 154.006]
 ['Logical' 123.131]
 ['Quant' 101.7]
 ['Domain' 28036.853]
 ['ComputerProgramming' 77.934]
 ['ElectronicsAndSemicon' -49.342]
 ['ComputerScience' -178.44]
 ['MechanicalEngg' 33.156]
 ['ElectricalEngg' -153.572]
 ['TelecomEngg' -64.832]
 ['CivilEngg' 146.868]
 ['conscientiousness' -19799.095]
 ['agreeableness' 15230.887]
 ['extraversion' 5054.964]
 ['nueroticism' -10592.207]
 ['openess_to_experience' -5725.161]]


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$